In [ ]:
from workalendar.europe import *
import requests
from datetime import datetime, timedelta, date
import json
import itertools
from typing import List
from typing_extensions import TypedDict, TypeAlias
from enum import Enum
import calendar

In [ ]:
calendarific_api_key = "2a7fb01b189f3d6735a2ab8d797b7a12dcba03d9"

def get_public_holidays_by_country(country_code):
    current_year = datetime.now().year
    current_year_str = [current_year - 1, current_year, current_year + 1]
    public_holidays = []

    try:
        for year in current_year_str:
            url = f"https://calendarific.com/api/v2/holidays?&api_key={calendarific_api_key}&country={country_code}&year={year}&type=national"
            res = requests.get(url)
            if (res.status_code == 200):
                public_holidays += [{'date': holiday['date']['datetime'], 'country': holiday['country']['id']} for holiday in res.json()['response']['holidays']]
            else:
                raise Exception(res.reason)
    except Exception as error:
        print(error)

    return public_holidays

In [ ]:
class DateDict(TypedDict, total=True):
  year: int
  month: int
  day: int

class ApiObject(TypedDict):
  date: DateDict
  country: str

class EventFrequency(Enum):
    Monthly = 'Monthly'
    Annually = 'Annually'
    Quaterly = 'Quarterly'
    SemiAnually = 'SemiAnually'

ApiObjectCollection: TypeAlias = List[ApiObject]

In [ ]:
def is_business_day(date: datetime, holidays: ApiObjectCollection) -> bool: 
    if date.weekday() >= 5:
        return False
    for holiday in holidays:
        if date.date() == datetime(**holiday['date']).date():
            return False
    return True

def find_business_date(date: datetime, public_holidays: ApiObjectCollection, days_before: int) -> datetime:
    count = 0
    while count < days_before:
        date -= timedelta(days=1)
        if is_business_day(date, public_holidays):
            count += 1

    return date

def find_common_business_date(event_cut_off_date: datetime, global_holidays: ApiObjectCollection):
    while True:
        if (is_business_day(event_cut_off_date, global_holidays)):
            return event_cut_off_date
        event_cut_off_date -= timedelta(days=1)

def find_valid_cut_off_date(cut_off_date: datetime, days_before: int, fund_origin_country: str, countries_involved: List[str] ):
    country_holidays_map = {}
    all_countries = [fund_origin_country] + countries_involved

    for country_code in all_countries:
        country_holidays_map[country_code] = get_public_holidays_by_country(country_code)

    countries_involved_holidays = list(itertools.chain(*[country_holidays_map[it] for it in all_countries]))

    fund_cut_off_date = find_business_date(cut_off_date, country_holidays_map[fund_origin_country], days_before)
    fund_common_cut_off_date = find_common_business_date(fund_cut_off_date, countries_involved_holidays)
    return fund_common_cut_off_date

def get_interval_dates(year: int, interval: EventFrequency, first_date=True):
    start_date = datetime(year, 1 ,1)
    _, end_day = calendar.monthrange(year, 12)
    end_date = datetime(year, 12, end_day)

    match interval:
        case EventFrequency.Quaterly:
            dates = []
            current_date = start_date
            while current_date <= end_date:
                if first_date:
                    dates.append(current_date)
                current_date += timedelta(days=91)
                if current_date <= end_date:
                    dates.append(current_date) if not first_date else None
            return dates

        case EventFrequency.Monthly:
            dates = [datetime(year, month, 1) for month in range(1, 13)]
            return dates if first_date else [datetime(year, month, calendar.monthrange(year, month)[1]) for month in range(1, 13)]

        case EventFrequency.Annually:
            return [start_date] if first_date else [end_date]

        case EventFrequency.SemiAnually:
            dates = []
            current_date = start_date
            while current_date <= end_date:
                if first_date:
                    dates.append(current_date)
                current_date += timedelta(days=182)
                if current_date <= end_date:
                    dates.append(current_date) if not first_date else None
            return dates
        case _:
            return []


# Expected result: Get a list dates
def find_available_dates(
        days_before: int, 
        countries_involved: List[str], 
        subscription_frequency: EventFrequency, 
        first_date: bool,
        year: int
):
    country_holidays_map = {}
    
    for country_code in countries_involved:
        country_holidays_map[country_code] = get_public_holidays_by_country(country_code)
    
    countries_involved_holidays = list(itertools.chain(*[country_holidays_map[it] for it in countries_involved]))

    dates = get_interval_dates(year=year, interval=subscription_frequency, first_date=first_date)
    print(dates)
    available_dates = []
    for date in dates:
        fund_cut_off_date = find_business_date(date=date, public_holidays=country_holidays_map[countries_involved[0]], days_before=days_before)
        available_dates.append(find_common_business_date(event_cut_off_date=fund_cut_off_date, global_holidays=countries_involved_holidays))
    print(available_dates)
    return available_dates


In [52]:
test_date = datetime(2023, 11, 13)
date = find_valid_cut_off_date(
    cut_off_date=test_date,
    days_before=1,
    fund_origin_country='HK',
    countries_involved=['US']
)
print('date lei')
print(date)

date lei
2023-11-09 00:00:00


In [58]:
dates = find_available_dates(
    days_before=7, 
    countries_involved=['HK', 'US'], 
    subscription_frequency=EventFrequency.Monthly, 
    first_date=True,
    year=2023   
)
print('date lei')
print(dates)

[datetime.datetime(2023, 1, 1, 0, 0), datetime.datetime(2023, 2, 1, 0, 0), datetime.datetime(2023, 3, 1, 0, 0), datetime.datetime(2023, 4, 1, 0, 0), datetime.datetime(2023, 5, 1, 0, 0), datetime.datetime(2023, 6, 1, 0, 0), datetime.datetime(2023, 7, 1, 0, 0), datetime.datetime(2023, 8, 1, 0, 0), datetime.datetime(2023, 9, 1, 0, 0), datetime.datetime(2023, 10, 1, 0, 0), datetime.datetime(2023, 11, 1, 0, 0), datetime.datetime(2023, 12, 1, 0, 0)]
[datetime.datetime(2022, 12, 20, 0, 0), datetime.datetime(2023, 1, 18, 0, 0), datetime.datetime(2023, 2, 17, 0, 0), datetime.datetime(2023, 3, 23, 0, 0), datetime.datetime(2023, 4, 20, 0, 0), datetime.datetime(2023, 5, 22, 0, 0), datetime.datetime(2023, 6, 21, 0, 0), datetime.datetime(2023, 7, 21, 0, 0), datetime.datetime(2023, 8, 23, 0, 0), datetime.datetime(2023, 9, 21, 0, 0), datetime.datetime(2023, 10, 20, 0, 0), datetime.datetime(2023, 11, 22, 0, 0)]
date lei
[datetime.datetime(2022, 12, 20, 0, 0), datetime.datetime(2023, 1, 18, 0, 0), datet